In [583]:
import pandas as pd
import numpy as np

In [584]:
cd Desktop\Springboard\assignments\Capstone Project 2

In [585]:
#load in datasets
fake = pd.read_csv('Fake.csv')
real = pd.read_csv('True.csv')

In [586]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [587]:
real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [588]:
#strip beginning of rows in real with LOCATION (Reuters) format

In [589]:
#starts with an upper case word
def uppercase(text):
    words = text.split()
    if len(words) >0:
        return words[0].isupper()
    else:
        return False
    
real['uppercase'] =  real.text.apply(first_upper)

In [590]:
#contains the string '(Reuters)'
real['reuters'] = real.text.str.contains('\(Reuters\)')

In [591]:
#contains an uppercase first word or reuters string
real['reuters_upper'] = (real.uppercase == True) | (real.reuters == True)

In [592]:
def strip_beginning(row, col1, col2):
    if row[col1]:
        return row[col2].partition('- ')[2]
    else:
        return row[col2]
real['text'] = real.apply(lambda x: strip_beginning(x,'reuters_upper', 'text'), axis=1)

In [593]:
real.drop(['uppercase', 'reuters', 'reuters_upper'], axis=1, inplace=True)

In [594]:
#create label column
fake['label']= 0
real['label'] = 1

In [595]:
#combine datasets
df = pd.concat([fake, real], ignore_index=True)

In [596]:
df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [597]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [619]:
#drop duplicates
dup = df[df.duplicated(['title', 'text'], keep=False)]

In [620]:
dup

,title,text,subject,date,label
9087,BARBRA STREISAND Gives Up On Dream Of Impeachi...,Barbra Streisand was an Obama sycophant and on...,politics,"Dec 31, 2017",0
9088,WATCH: SENATOR LINDSEY GRAHAM DROPS BOMBSHELL…...,Everyone suspected the sketchy Steele Dossier ...,politics,"Dec 31, 2017",0
9089,“CONSERVATIVE GAY GUY” BLASTS Pence’s Aspen Ne...,It s been said that good fences make good neig...,politics,"Dec 30, 2017",0
9091,BILL NYE The FAKE Science Guy THREATENS Conser...,"Friday on MSNBC, climate activist Bill Nye war...",politics,"Dec 30, 2017",0
9093,EMBARRASSING: Anti-Trump “THE HILL” Gets SLAMM...,"#JokeNewsAfter record, cold temperatures were ...",politics,"Dec 30, 2017",0
...,...,...,...,...,...
44771,"Europeans, Africans agree renewed push to tack...",Europe s big four continental powers and thr...,worldnews,"August 28, 2017",1
44826,Thailand's ousted PM Yingluck has fled abroad:...,Ousted Thai prime minister Yingluck Shinawatra...,worldnews,"August 25, 2017",1
44834,Thailand's ousted PM Yingluck has fled abroad:...,Ousted Thai prime minister Yingluck Shinawatra...,worldnews,"August 25, 2017",1
44887,"U.S., North Korea clash at U.N. forum over nuc...",North Korea and the United States clashed at a...,worldnews,"August 22, 2017",1


In [625]:
df.drop(dup.index, inplace=True)

In [627]:
df.reset_index(drop=True)

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
33795,'Fully committed' NATO backs new U.S. approach...,NATO allies on Tuesday welcomed President Dona...,worldnews,"August 22, 2017",1
33796,LexisNexis withdrew two products from Chinese ...,"LexisNexis, a provider of legal, regulatory an...",worldnews,"August 22, 2017",1
33797,Minsk cultural hub becomes haven from authorities,In the shadow of disused Soviet-era factories ...,worldnews,"August 22, 2017",1
33798,Vatican upbeat on possibility of Pope Francis ...,Vatican Secretary of State Cardinal Pietro Par...,worldnews,"August 22, 2017",1


In [628]:
df.to_csv('combined.csv')